In [1]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import torch
import torch.nn as nn

In [2]:
#path = "/content/sample_data/activity_monitoring/"
path = 'D:\\University\\Machine learning\\project\\'

In [3]:
train_msAcc = np.load(path+"training\\"+"trainMSAccelerometer.npy")
train_msGyro = np.load(path+"training\\"+"trainMSGyroscope.npy")
train_labels = np.load(path+"training\\"+"trainlabels.npy")

test_msAcc = np.load(path+"testing\\"+"testMSAccelerometer.npy")
test_msGyro = np.load(path+"testing\\"+"testMSGyroscope.npy")
test_labels = np.load(path+"testing\\"+"testlabels.npy")



In [4]:
print(train_msAcc.shape)
print(train_msGyro.shape)
print(train_labels.shape)

print(test_msAcc.shape)
print(test_msGyro.shape)
print(test_labels.shape)


(2284, 268, 3)
(2284, 268, 3)
(2284,)
(2288, 268, 3)
(2288, 268, 3)
(2288,)


In [5]:
train_num_examples = np.shape(train_labels)[0]
test_num_examples = np.shape(test_labels)[0]
print("Num Train Examples: ", train_num_examples)
print("Num Test Examples: ", train_num_examples)

Num Train Examples:  2284
Num Test Examples:  2284


In [6]:
num_features = 4
train_features = np.zeros((train_num_examples, 2*num_features, 3))
test_features = np.zeros((test_num_examples, 2*num_features, 3))

print(np.shape(train_features))
print(np.shape(test_features))


(2284, 8, 3)
(2288, 8, 3)


In [7]:
#feaure extraction (mean, max, min, std)
# generation of features for training data set
train_features[:,0,:] = np.mean(train_msAcc, axis=1)
train_features[:,1,:] = np.max(train_msAcc, axis=1)
train_features[:,2,:] = np.min(train_msAcc, axis=1)
train_features[:,3,:] = np.std(train_msAcc, axis=1)

train_features[:,4,:] = np.mean(train_msGyro, axis=1)
train_features[:,5,:] = np.max(train_msGyro, axis=1)
train_features[:,6,:] = np.min(train_msGyro, axis=1)
train_features[:,7,:] = np.std(train_msGyro, axis=1)
#print(train_features[:,3,:])


# generation of features for testing data set
test_features[:,0,:] = np.mean(test_msAcc, axis=1)
test_features[:,1,:] = np.max(test_msAcc, axis=1)
test_features[:,2,:] = np.min(test_msAcc, axis=1)
test_features[:,3,:] = np.std(test_msAcc, axis=1)

test_features[:,4,:] = np.mean(test_msGyro, axis=1)
test_features[:,5,:] = np.max(test_msGyro, axis=1)
test_features[:,6,:] = np.min(test_msGyro, axis=1)
test_features[:,7,:] = np.std(test_msGyro, axis=1)


In [8]:
print(np.shape(train_features))
train_features_reshaped = np.reshape(train_features, (np.shape(train_features)[0], np.shape(train_features)[1]*np.shape(train_features)[2] ))
print(np.shape(train_features_reshaped))


print(np.shape(test_features))
test_features_reshaped = np.reshape(test_features, (np.shape(test_features)[0], np.shape(test_features)[1]*np.shape(test_features)[2] ))
print(np.shape(test_features_reshaped))



(2284, 8, 3)
(2284, 24)
(2288, 8, 3)
(2288, 24)


In [9]:
#classification using SVM
classfier = SVC(C=1.0, kernel='linear')  # non-linear kernel is proposed by the fernando's paper
classfier.fit(train_features_reshaped,train_labels)
estimatedLabels = classfier.predict(test_features_reshaped)

accuracy = accuracy_score(test_labels,estimatedLabels)
weightedF1 = f1_score(test_labels,estimatedLabels,average='weighted')
averageF1 = f1_score(test_labels,estimatedLabels,average='macro')
allF1Scores = f1_score(test_labels,estimatedLabels,average=None)
conf_matrix = confusion_matrix(test_labels,estimatedLabels)

# Print results
print('   C = 1.0 ')
print('   Average F1-score = %.4f' % (averageF1))
print('   Test accuracy = %.2f %%' % (accuracy*100))
print('   Weighted F1-score = %.4f' % (weightedF1))
print('   All F1-scores:')
print(allF1Scores)
print('   Confusion Matrix:')
print(conf_matrix)

print('-------------------------------------------------------')



   C = 1.0 
   Average F1-score = 0.3485
   Test accuracy = 35.45 %
   Weighted F1-score = 0.3496
   All F1-scores:
[0.775      0.60606061 0.4        0.5        0.15841584 0.22857143
 0.44897959 0.24719101 0.24175824 0.10810811 0.32432432 0.27118644
 0.8        0.27826087 0.63529412 0.33333333 0.4742268  0.38235294
 0.28571429 0.35443038 0.1509434  0.23333333 0.37974684 0.16129032
 0.18390805 0.34146341 0.36363636 0.14285714 0.14285714 0.16494845
 0.25806452 0.2970297  0.275      0.31325301 0.23404255 0.23728814
 0.64197531 0.25806452 0.54347826 0.34693878 0.48192771 0.45454545
 0.22580645 0.35087719 0.7628866  0.09876543 0.30434783 0.25641026
 0.35514019 0.51282051 0.52272727 0.55813953 0.09836066 0.54545455
 0.11650485]
   Confusion Matrix:
[[31  0  0 ...  0  0  0]
 [ 1 30  4 ...  0  0  0]
 [ 0  2 15 ...  0  2  0]
 ...
 [ 0  1  1 ...  3  0  0]
 [ 0  0  0 ...  0 21  0]
 [ 0  0  0 ...  0  0  6]]
-------------------------------------------------------


In [ ]:
#neural network 
# batchSize = 50
# learningRate = 0.5 # NOTE: tested: 1, 0.5, 0.1, 0.01
# epochs = 150
# targetDataProportion = 1

# msbandModel = {
#     'name': 'CNN',
#     'nb_conv_blocks' : 3,
#     'nb_conv_kernels' : [10,10,10],
#     'conv_kernels_size' : [(9,),(11,),(11,)],
#     'pooling_size' : [(2,),(2,),(2,)],
#     'activation' : 'relu',
#     }
# denseSize = [2000,2000,2000]
# denseActivation = 'relu'



# trainingMSAcc = torch.from_numpy(train_msAcc)
# trainingMSGyro = torch.from_numpy(train_msGyro)
# trainingLabels = torch.from_numpy(train_labels)

# testingMSAcc = torch.from_numpy(test_msAcc)
# testingMSGyro = torch.from_numpy(test_msGyro)
# testingLabels = torch.from_numpy(test_labels)



In [ ]:
# #feaure extraction (mean, max, min, std)
# m = np.mean(train_msAcc, axis=1)
# print(np.mean(train_msAcc[0,:,0]))
# print(np.mean(train_msAcc[0,:,1]))
# print(np.mean(train_msAcc[0,:,2]))
# print(m[0])
# print(m[1])
# print(m.shape)
# m = m.flatten()
# print(m.shape)
# print(m[0])
# print(m[1])
# print(m[2])
# print(m[3])

0.5609969
0.36283487
0.7257417
[0.5609969  0.36283487 0.7257417 ]
[ 0.9604474  -0.05970696  0.23163207]
(87, 3)
(261,)
0.5609969
0.36283487
0.7257417
0.9604474
